In [2]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split

In [3]:
#read the csv
#column Breed shows the y with the image name
data_csv = pd.read_csv('dataset_CV.csv')
# data_csv

In [4]:
# preprocessing
trans = transforms.Compose([
    transforms.Resize((80, 80)),
    transforms.ToTensor(),
    transforms.Normalize(mean=0.5, std=0.5),  #values anpassen
    #transforms.AugMix(),
])
location = r"dog_breed_photos"

def create_data(path):
    image = Image.open(Path(location) / path)
    #img = image.int()
    img = trans(image)
    image.close()
    return img

from sklearn import preprocessing
import torch

data_csv['data'] = data_csv['Image'].apply(lambda image: create_data(image))
# data_csv

In [5]:
#creat dataframe for loader
df = pd.concat([data_csv['data'], data_csv['Breed']], ignore_index=True, axis=1)

In [6]:

labels = ["AMERICAN STAFFORDSHIRE BULL TERRIER", "BULL ARAB", "GERMAN SHEPHERD", "SIBERIAN HUSKY", "BORDER COLLIE"]

df[1] = df[1].apply(labels.index)

In [7]:
df.head()

,0,1
0,"[[[tensor(0.1451), tensor(0.1922), tensor(0.12...",3
1,"[[[tensor(-0.0353), tensor(-0.0745), tensor(-0...",3
2,"[[[tensor(0.0824), tensor(0.3176), tensor(0.56...",3
3,"[[[tensor(0.3255), tensor(0.2941), tensor(0.32...",3
4,"[[[tensor(-0.7647), tensor(-0.6157), tensor(-0...",3


In [8]:
# train, test split
train, test = train_test_split(df, test_size=0.25, random_state=42, shuffle=True)
train

,0,1
1095,"[[[tensor(0.3098), tensor(-0.2784), tensor(-0....",0
1138,"[[[tensor(0.2078), tensor(0.1765), tensor(0.16...",1
1612,"[[[tensor(0.2078), tensor(0.4431), tensor(0.49...",3
2080,"[[[tensor(1.), tensor(1.), tensor(0.9843), ten...",3
3897,"[[[tensor(-0.0745), tensor(-0.0824), tensor(-0...",4
...,...,...
3444,"[[[tensor(0.6863), tensor(0.7098), tensor(0.72...",3
466,"[[[tensor(-0.0275), tensor(-0.0039), tensor(-0...",0
3092,"[[[tensor(-0.5137), tensor(-0.5608), tensor(-0...",3
3772,"[[[tensor(0.5686), tensor(0.5922), tensor(0.63...",0


In [9]:
# #dataloader
# train_loader = torch.utils.data.DataLoader(train, batch_size=4, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test, batch_size=4, shuffle=True)

In [25]:
#Define model

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(289, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [12]:
train = train.to_numpy()

In [24]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for inputs, labels in train:
        labels = torch.tensor(labels, dtype=torch.int8)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        print(f"current epoch: {epoch + 1}")

print('Finished Training')

IndexError: Dimension out of range (expected to be in range of [-3, 2], but got 3)

In [ ]:
#eval

In [ ]:
#visualize

In [ ]:
#plot muss noch angepasst werden
plt.rcParams["savefig.bbox"] = 'tight'
orig_img = df[42]
# if you change the seed, make sure that the randomly-applied transforms
# properly show that the image can be both transformed and *not* transformed!
torch.manual_seed(0)


def plot(imgs, with_orig=True, row_title=None, **imshow_kwargs):
    if not isinstance(imgs[0], list):
        # Make a 2d grid even if there's just 1 row
        imgs = [imgs]

    num_rows = len(imgs)
    num_cols = len(imgs[0]) + with_orig
    fig, axs = plt.subplots(nrows=num_rows, ncols=num_cols, squeeze=False)
    for row_idx, row in enumerate(imgs):
        row = [orig_img] + row if with_orig else row
        for col_idx, img in enumerate(row):
            ax = axs[row_idx, col_idx]
            ax.imshow(np.asarray(img), **imshow_kwargs)
            ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

    if with_orig:
        axs[0, 0].set(title='Original image')
        axs[0, 0].title.set_size(8)
    if row_title is not None:
        for row_idx in range(num_rows):
            axs[row_idx, 0].set(ylabel=row_title[row_idx])

    plt.tight_layout()